##### Studi Kasus 1

In [ ]:
import requests
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

url = "http://103.82.92.37/DM20232/getdata.php?table=ms_lulusan"

payload={}
headers = {'api-key': 'DM20232UBP'}

response = requests.request("GET", url, headers=headers, data=payload)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data['data'])

    jumlah_data = len(df)
    print("Data yang diterima: ", jumlah_data, '\n')
    print(data)

else:
    print(f"Gagal mengambil data. Status code: {response.status_code}: {response.text}")

In [ ]:
import pandas as pd # type: ignore
data = pd.read_csv('dataset/ms_lulusan.csv')

In [ ]:
import pandas as pd

for mahasiswa in data:
    # Mendapatkan tanggal masuk
    tgl_masuk = pd.to_datetime(mahasiswa['tgl_masuk'])
    tanggal_lulus = pd.to_datetime(mahasiswa['tanggal_lulus'])

    lama_kuliah = tanggal_lulus - tgl_masuk

    tahun = lama_kuliah.days // 365
    bulan = (lama_kuliah.days % 365) // 30

    if tahun <= 4:
        print("NIM = ", mahasiswa['nim'], "- Lama kuliah = ", tahun, "tahun", bulan, "bulan")
    else:
        print("NIM = ", mahasiswa['nim'], "- Lama kuliah = ", tahun, "tahun", bulan, "bulan (lulus tidak tepat waktu)")


In [ ]:
import pandas as pd

# Membuat list untuk menyimpan durasi studi setiap mahasiswa
durasi_studi = []

# Melakukan iterasi pada data untuk menghitung durasi studi setiap mahasiswa
for mahasiswa in data:
    # Mengonversi tanggal_masuk dan tanggal_lulus menjadi objek datetime
    tgl_masuk = pd.to_datetime(mahasiswa['tgl_masuk'])
    tanggal_lulus = pd.to_datetime(mahasiswa['tanggal_lulus'])

    # Menghitung durasi studi dalam bulan
    lama_kuliah_bulan = (tanggal_lulus.year - tgl_masuk.year) * 12 + (tanggal_lulus.month - tgl_masuk.month)
    durasi_studi.append(lama_kuliah_bulan)

# Mengonversi list durasi studi menjadi Series pandas
durasi_studi_series = pd.Series(durasi_studi)

# Menghitung rata-rata, median, dan modus
rata_rata_durasi = durasi_studi_series.mean()
median_durasi = durasi_studi_series.median()
modus_durasi = durasi_studi_series.mode()[0]

# Konversi durasi rata-rata, median, dan modus menjadi tahun dan bulan
tahun_rata_rata = int(rata_rata_durasi) // 12
bulan_rata_rata = int(rata_rata_durasi) % 12

tahun_median = int(median_durasi) // 12
bulan_median = int(median_durasi) % 12

tahun_modus = int(modus_durasi) // 12
bulan_modus = int(modus_durasi) % 12

# Menampilkan hasil
print("Rata-rata lama kuliah:", tahun_rata_rata, "tahun", bulan_rata_rata, "bulan")
print("Median lama kuliah:", tahun_median, "tahun", bulan_median, "bulan")
print("Modus lama kuliah:", tahun_modus, "tahun", bulan_modus, "bulan")

In [ ]:
import matplotlib.pyplot as plt

# Histogram
plt.figure(figsize=(10, 6))
plt.hist(durasi_studi, bins=range(min(durasi_studi), max(durasi_studi) + 1, 1), edgecolor='black')
plt.title('Distribusi Durasi Studi Mahasiswa')
plt.xlabel('Durasi Studi (bulan)')
plt.ylabel('Frekuensi')
plt.grid(True)
plt.show()

In [ ]:
# Pie Chart Menghitung jumlah mahasiswa dalam setiap kategori durasi studi per bulan
kategori_durasi_bulan = pd.cut(durasi_studi_series, bins=range(0, durasi_studi_series.max() + 1, 1), labels=range(1, durasi_studi_series.max() + 1))
jumlah_mahasiswa_per_kategori_bulan = kategori_durasi_bulan.value_counts().sort_index()

# Membuat pie chart
plt.figure(figsize=(8, 8))
plt.pie(jumlah_mahasiswa_per_kategori_bulan, labels=jumlah_mahasiswa_per_kategori_bulan.index, autopct='%1.1f%%', startangle=140)
plt.title('Pie Chart Durasi Studi Mahasiswa (per Bulan)')
plt.show()

##### Studi Kasus 2

In [ ]:
import requests
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

url = "http://103.82.92.37/DM20232/getdata.php?table=ms_lulusan"

payload={}
headers = {'api-key': 'DM20232UBP'}

response = requests.request("GET", url, headers=headers, data=payload)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data['data'])

    jumlah_data = len(df)
    print("Data yang diterima: ", jumlah_data)

else:
    print(f"Gagal mengambil data. Status code: {response.status_code}: {response.text}")


In [ ]:
print("Daftar data lulusan")
df

In [ ]:
# 1. Penanganan nilai hilang (missing value)

df_missing = df.isnull()
jumlah_hilang_perkolom = df_missing.sum(axis=0)
pd.DataFrame(jumlah_hilang_perkolom)

In [ ]:
# 2. Identifikasi baris yang merupakan duplikat dari dataset

duplicate_value = df.duplicated().sum()
duplicate_value

In [ ]:
# 3. Identifikasi dan penanganan outliers
# Menambahkan outlier ke kolom 'tahun_lahir' menggunakan z-score
column = 'tahun_lahir'
z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())

threshold = 3
outliers = df[z_scores > threshold]

# Menampilkan jumlah outlier yang ditambahkan
print("Jumlah outliers yang ditambahkan:", len(outliers))

# Menampilkan 10 data outlier
print("Data outlier:")
print(pd.DataFrame(outliers))

import matplotlib.pyplot as plt

# Visualisasi box plot untuk kolom 'tahun_lahir' dengan menandai outlier
plt.figure(figsize=(10, 6))
plt.boxplot(df['tahun_lahir'], vert=False, widths=0.7, patch_artist=True, meanline=True, showmeans=True)
plt.title('Boxplot Tahun Lahir')
plt.xlabel('Tahun Lahir')
plt.show()

In [ ]:
# 4. Identifikasi dan penanganan kesalahan tipe atau ketidaksesuaiaan format

# Menampilkan tipe data untuk setiap kolom
print("Tipe data untuk setiap kolom")
print(df.dtypes)

try:
    df['tanggal_lulus'] = pd.to_datetime(df['tanggal_lulus'])
    df['tgl_masuk'] = pd.to_datetime(df['tgl_masuk'])
    df['status_masuk'] = pd.Categorical(df['status_masuk'])
    df['status_pegawai'] = pd.Categorical(df['status_pegawai'])
except ValueError as e:
    print("Kesalahan konversi tipe data ", e)

# Menampilkan tipe data setelah penanganan
    print("\nTipe data setelah penanganan")
print(df.dtypes)

In [ ]:
# 5. Mengidentifikasi dan menangani anomali atau nilai yang tidak sesuai dengan aturan bisnis atau ilmu pengetahuan

# Menampilkan jumlah data sebelum penanganan 
jumlah_data_sebelum = len(df)
print('jumlah data sebelum penanganan: ', jumlah_data_sebelum)

# Mengganti nilai tahun lahin yang tidak sesuai dengan NaN
tahun_min = 1900
tahun_max = pd.Timestamp.now().year
df.loc[(df['tahun_lahir'] < tahun_min) | (df['tahun_lahir'] > tahun_max), 'tahun_lahir'] = None

# Menampilkan jumlah data setelah penanganan 
jumlah_data_setelah = len(df)
print('jumlah data setelah penanganan: ', jumlah_data_setelah)

# Menampilkan 5 baris pertama DataFrame setelah penanganan
print("\n5 baris pertama DataFrame setelah penanganan: ")
print(df.head())


In [ ]:
# 6. Membersihkan dan memproses data teks, termasuk penghapusan karakter atau stemming

# Download paket 'punkt' untuk tokenisasi kata dan 'wordnet' untuk stemming
nltk.download('punkt')
nltk.download('wordnet')

# Inisialisasi WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Fungsi untuk membersihkan teks
def cleanText(text):
    text = str(text).lower() # ubah menjadi huruf kecil
    text = text.translate(str.maketrans('', '', string.punctuation)) # hapus karakter khusus
    tokens = word_tokenize(text) # tokenisasi kata
    return tokens


# Fungsi untuk melakukan stemming pada kata
def stem_words(words):
    stemmed_words = [lemmatizer.lemmatize(word) for word in words]
    return stemmed_words

# Membersihkan dan memproses data teks pada semua kolom teks 
for column in df.select_dtypes(include=['object']).columns:
    df[column + '_cleaned'] = df[column].apply(cleanText)
    df[column + '_stemmed'] = df[column + '_cleaned'].apply(stem_words)

print("5 baris pertama DataFrame setelah pemrosesan: ")
print(df.head())


##### Studi Kasus 3

In [ ]:
import pandas as pd

data_lulusan = pd.read_csv('dataset/ms_lulusan.csv')

df_lulusan = pd.DataFrame(data_lulusan)
df_lulusan

In [ ]:
data_transkip = pd.read_excel('dataset/transkip_nilai.xlsx')
df_transkip = pd.DataFrame(data_transkip)
df_transkip

In [ ]:
df_merge_transkip = pd.merge(df_lulusan, df_transkip, on='nim', how='left')
df_merge_transkip.head(100)

In [ ]:
def hitung_durasi_studi(df):
    
    # Konversi kolom tanggal_lulus dan tgl_masuk kedalam tipe data datetime
    df_lulusan['tanggal_lulus'] = pd.to_datetime(df_lulusan['tanggal_lulus'])
    df_lulusan['tgl_masuk'] = pd.to_datetime(df_lulusan['tgl_masuk'])

    # Kelompokkan data berdasarkan NIM
    grouped = df.groupby('nim')

    durasi_studi_data = []
    
    for (nim), group in grouped:
        # Hitung nilai total (jumlah perkalian nilai dengan SKS) dan jumlah total SKS
        durasi_studi = (group['tanggal_lulus'] - group['tgl_masuk']).sum()
        # Ambil tahun, bulan, dan hari dari objek timedelta
        durasi_studi_years = durasi_studi.days // 365
        
        # Tambahkan data IPS ke list
        durasi_studi_str = f"{durasi_studi_years} tahun"
        durasi_studi_data.append({'nim': nim, 'durasi_studi': durasi_studi_str})
    
    # Buat DataFrame dari list data IPS
    df_durasi_studi = pd.DataFrame(durasi_studi_data)
    return df_durasi_studi

# Hitung IPS untuk data transkrip nilai
durasi_studi = hitung_durasi_studi(df_lulusan)

# Tampilkan hasil
df_durasi_studi = pd.DataFrame(durasi_studi)
df_durasi_studi.drop_duplicates()

df_durasi_studi

In [ ]:
df_merge_durasi_studi = pd.merge(df_merge_transkip, df_durasi_studi, on='nim', how='left')
df_merge_durasi_studi

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.hist(df_merge_durasi_studi['durasi_studi'], bins=20, color='skyblue', edgecolor='black')
plt.title("Distribusi Durasi Studi Mahasiswa")
plt.xlabel("Durasi Studi (tahun)")
plt.ylabel("Jumlah Mahasiswa")
plt.grid(True)
plt.show()

In [ ]:
df_merge_transkip = pd.merge( df_lulusan )

In [ ]:
df_lulusan.isnull().sum()

In [ ]:
df['tanggal_lulus'] = pd.to_datetime(df['tanggal_lulus'])
df['tgl_masuk'] = pd.to_datetime(df['tgl_masuk'])
df['durasi_studi'] = (df['tanggal_lulus'] - df['tgl_masuk']).dt.days // 365

# Analisis berdasarkan faktor-faktor yang diberikan
faktor_analisis = ['prodi', 'jenis_kelamin', 'tahun_lahir', 'status_pegawai']

for faktor in faktor_analisis:
    print(f"Analisis berdasarkan {faktor}")
    print(df.groupby(faktor)['durasi_studi'].mean())
    print("\n")